- nbp: Carbon Mass Flux out of Atmosphere Due to Net Biospheric Production on Land [kgC m-2 s-1] [kg m-2 s-1]
- fgco2: Surface Downward Mass Flux of Carbon as CO2 [kgC m-2 s-1] [kg m-2 s-1]
- ra: Carbon Mass Flux into Atmosphere Due to Autotrophic (Plant) Respiration on Land [kgC m-2 s-1] [kg m-2 s-1]
- rh: Total Heterotrophic Respiration on Land as Carbon Mass Flux [kgC m-2 s-1] [kg m-2 s-1]
- gpp: Carbon Mass Flux out of Atmosphere Due to Gross Primary Production on Land [kgC m-2 s-1] [kg m-2 s-1]
- fFire: Carbon Mass Flux into Atmosphere Due to CO2 Emission from Fire Excluding Land-Use Change [kgC m-2 s-1] [kg m-2 s-1]

In [1]:
# to set
r_sulfur =  ['12', 
             '13', 
             '14']
r_over =  ['1', 
           '2', 
           '3'
          ]
end_I = 2062
end_II = 2150
end_III = 2249

In [2]:
%matplotlib inline
import xarray as xr
import os
import numpy as np
import netCDF4
from netCDF4 import Dataset
import pandas as pd
import nc_time_axis 
import cftime
import itertools

from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit 

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import proplot as pplt

import warnings
warnings.filterwarnings('ignore')

/data/softs/anaconda3-2020.07/envs/gloenv_py3.9/lib/python3.9/site-packages/pyproj/__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


In [3]:
def check_time_bounds(data:xr.Dataset):
    return data.drop('time_bounds') if 'time_bounds' in data.data_vars else data

def check_lat_lon(data):
    if 'longitude' in data.dims:
        data = data.rename({'longitude': 'lon'})
    if 'latitude' in data.dims:
        data = data.rename({'latitude': 'lat'})
    return data


# area weighting and average calcualtion
def field_avg_2(data):
    import numpy
    import xarray
    if isinstance(data, xarray.Dataset):
        _das = [data.get(_var) for _var in data.data_vars]; _to_ds = True;
    elif isinstance(data, xarray.DataArray):
        _das = [data]; _to_ds = False;
    else:
        raise TypeError('Input data type should either be a Dataset or a DataArray from the xarray module.')
    _weights = numpy.cos(numpy.deg2rad(data.lat)); _weights.name = 'weights';
    _avgs = []
    for _da in _das:
        _avgs.append(check_lat_lon(_da).weighted(_weights).mean(('lon', 'lat'), keep_attrs = True))
    return xarray.merge(_avgs) if _to_ds else _avgs[0]

def change_T_unit(df):
## turn Fahrenheit? to degC
    try:
        var = 'tas'
        if df[var].min().values > 40 and df[var].max().values > 60:
            df[var] = df[var] - 273.15
    except:
        df = df
    return df

In [4]:
dir_NER = '..'
dir_hist = f'{dir_NER}historical/'
dir_piControl=f'{dir_NER}piControl/'
dir_sulfur = f'{dir_NER}ssp534-sulfur/'
dir_over = f'{dir_NER}ssp534-over/'

In [5]:
scens = ['ssp534-sulfur', 'ssp534-over']
params = ['fgco2', 'nbp', 'rh', 'ra', 'gpp', 'fFire']
nb_cs_params = len(params) 

In [6]:
#load in data
# turn from monthly mean to yearly average
# merge with historical data for smooth 10 year rolling mean
data = dict()
over=dict()
over_var=dict()
sulf=dict()
sulf_var=dict()
    
for exp in ['ssp126', 'ssp534-over', 'historical', 'ssp534-sulfur']:
    if exp == 'ssp534-over':
        ds =[]
        for i in range(0, nb_cs_params):
            for mem in r_over:
                over[f'{params[i]}_{mem}'] = xr.open_mfdataset(f'{dir_over}CNRM-ESM2-1_ssp534-over_r{mem}i1p1f2/merged/{params[i]}_*mon_CNRM-ESM2-1_{exp}_r{mem}i1p1f2_*_201501-224912.nc'
                                           , engine="netcdf4").groupby('time.year').mean('time')
            data[f'{params[i]}_{exp}'] = xr.concat([over[f'{params[i]}_1'], over[f'{params[i]}_2'], over[f'{params[i]}_3']], dim='member')
        
    
# load in historic data to get smooth 10year rolling mean at the start of the future simulations
    elif exp == 'historical':
        ds =[]
        for i in range(0, nb_cs_params):
            data[f'{params[i]}_{exp}'] = xr.open_mfdataset(f'{dir_NER}historical/{params[i]}_*_g*_185001-201412.nc'
                                    , engine="netcdf4").drop('time_bounds').groupby('time.year').mean('time')

    elif exp == 'ssp534-sulfur':
        ds=[]
        for i in range(0, nb_cs_params):
            for mem in r_sulfur:
                sulf[f'{params[i]}_{mem}'] = xr.open_mfdataset(f'{dir_sulfur}CNRM-ESM2-1_ssp534-over_r{mem}i1p1f2*/merged/{params[i]}_*mon_CNRM-ESM2-1_{exp}_r{mem}i1p1f2_*_201501-224912.nc'
                                           , engine="netcdf4").groupby('time.year').mean('time')
            data[f'{params[i]}_{exp}'] = xr.concat([sulf[f'{params[i]}_12'], sulf[f'{params[i]}_13'], sulf[f'{params[i]}_14']], dim='member')

        

# data['ssp534'] = xr.merge([over_ssp534, hist])
# data['sulfur']= xr.merge([over_sulfur, hist])



In [7]:
# piControl

data['fgco2_piControl'] = xr.open_mfdataset(f'{dir_piControl}fgco2_Omon_*_g*_185001-234912.nc'
                                                             , engine="netcdf4").mean('time')
data['nbp_piControl'] = xr.open_mfdataset(f'{dir_piControl}nbp_Lmon_*_g*_185001-234912.nc'
                                                             , engine="netcdf4").mean('time')


# area data
areacello = xr.open_mfdataset(f'/archive/globc/baur/CNRM_runs/areacello*.nc'
                                                             , engine="netcdf4")
areacella = xr.open_mfdataset(f'/archive/globc/baur/CNRM_runs/areacella*.nc'
                                                             , engine="netcdf4")
sftlf =  xr.open_dataset('/data/scratch/globc/baur/RE_analysis/LUC_file_IMAGE/sftlf_fx_CNRM-ESM2-1_historical_r2i1p1f2_gr.nc',
                                    engine="netcdf4")

#### Add historical for smooth rolling mean

In [8]:
for scen in scens:
    for param in params:
        data[f'{param}_{scen}_long'] = xr.merge([data[f'{param}_{scen}'], data[f'{param}_historical']])

In [9]:
SAI = pd.read_csv('/data/scratch/globc/baur/NER_analysis/data/aod_times2_5.csv').drop({'Unnamed: 0'}, axis=1)

In [10]:
co2_fut = xr.open_mfdataset(f'{dir_sulfur}CNRM-ESM2-1_ssp534-over_r12i1p1f2*/merged/co2_Amon_*_g*_201501-224912.nc'
                            , engine="netcdf4").groupby('time.year').mean('time')

co2_ = co2_fut.sel(plev=slice(100000, 70000)).mean('plev').mean(('lat', 'lon')) * 1000000

In [11]:
# Calculate sink in GtC/yr
land=dict()
ocean=dict()
gpp=dict()
ra=dict()
rh=dict()
fire=dict()


for scen in scens:
    land[scen] = (areacella['areacella'].where(sftlf['sftlf']>50)*data[f'nbp_{scen}_long']*3600*365*24/1e12).sum(('lat', 'lon')).rolling(year=10).mean().sel(year=slice(2015,2249)).load()
    ocean[scen] = (areacello['areacello']*data[f'fgco2_{scen}_long']*3600*365*24/1e12).sum(('y', 'x')).rolling(year=10).mean().sel(year=slice(2015,2249)).load()   
    gpp[scen] = (areacella['areacella'].where(sftlf['sftlf']>50)*data[f'gpp_{scen}_long']*3600*365*24/1e12).sum(('lat', 'lon')).rolling(year=10).mean().sel(year=slice(2015,2249)).load()
    ra[scen] = (areacella['areacella'].where(sftlf['sftlf']>50)*data[f'ra_{scen}_long']*3600*365*24/1e12).sum(('lat', 'lon')).rolling(year=10).mean().sel(year=slice(2015,2249)).load()
    rh[scen] = (areacella['areacella'].where(sftlf['sftlf']>50)*data[f'rh_{scen}_long']*3600*365*24/1e12).sum(('lat', 'lon')).rolling(year=10).mean().sel(year=slice(2015,2249)).load()
    fire[scen] = (areacella['areacella'].where(sftlf['sftlf']>50)*data[f'fFire_{scen}_long']*3600*365*24/1e12).sum(('lat', 'lon')).rolling(year=10).mean().sel(year=slice(2015,2249)).load()
    
    
    
land['piControl'] = (areacella['areacella'].where(sftlf['sftlf']>50)*data[f'nbp_piControl']*3600*365*24/1e12).sum(('lat', 'lon')).load()
ocean['piControl'] = (areacello['areacello']*data[f'fgco2_piControl']*3600*365*24/1e12).sum(('y', 'x')).load()    

In [12]:
# subtract piControl sink from sinks -> to only get anthropogenic sinks/sources
land_pi=dict()
ocean_pi=dict()

for scen in ['ssp534-over', 'ssp534-sulfur']:
    land_pi[scen] = land[scen]-land['piControl']
    ocean_pi[scen] = ocean[scen]-ocean['piControl']

# Cumulative sink values in the 3 phases

In [21]:
phase_I=dict()
phase_II=dict()
phase_III=dict()

cum_C = dict()
C_diff = dict()

main_list=[]


for scen in scens:
    for i in ['land', 'ocean']:
        if i == 'land':
            df = land_pi
            var = 'nbp'
        elif i == 'ocean':
            df = ocean_pi
            var = 'fgco2'
        
        phase = dict()
        phase['I'] = df[scen].sel(year=slice(2015, end_I))
        phase['II'] = df[scen].sel(year=slice(end_I, end_II))
        phase['III'] = df[scen].sel(year=slice(end_II, end_III))
        
        for P in ['I', 'II', 'III', 'all']:

            for m in list(range(0,2+1)):
                if P == 'all': #total over all phases
                    cum_C[f'{scen}_{i}_{P}_{m}'] = round(df[scen][var].sel(member=m).sum().values.item(), 1)
                else:
                    cum_C[f'{scen}_{i}_{P}_{m}'] = round(phase[P][var].sel(member=m).sum().values.item(), 1)
                    

    # get the sum of land and ocean
    for m in list(range(0,2+1)):
        cum_C[f'{scen}_total_I_{m}'] = round(land_pi[scen]['nbp'].sel(year=slice(2015, end_I)).sel(member=m).sum().values.item() + ocean_pi[scen]['fgco2'].sel(year=slice(2015, end_I)).sel(member=m).sum().values.item(), 1)
        cum_C[f'{scen}_total_II_{m}'] = round(land_pi[scen]['nbp'].sel(year=slice(end_I, end_II)).sel(member=m).sum().values.item() + ocean_pi[scen]['fgco2'].sel(year=slice(end_I, end_II)).sel(member=m).sum().values.item(), 1)
        cum_C[f'{scen}_total_III_{m}'] = round(land_pi[scen]['nbp'].sel(year=slice(end_II, end_III)).sel(member=m).sum().values.item() + ocean_pi[scen]['fgco2'].sel(year=slice(end_II, end_III)).sel(member=m).sum().values.item(), 1)
        cum_C[f'{scen}_total_all_{m}'] = round(land_pi[scen]['nbp'].sel(member=m).sum().values.item() + ocean_pi[scen]['fgco2'].sel(member=m).sum().values.item(), 1)

            
a = [0,1,2]
b = [0,1,2]
          
            
            
for P in ['I', 'II', 'III', 'all']:
    
    for i in ['land', 'ocean', 'total']: 
        print(i)
        
        stat_list = []
    
        for m in list(range(0,2+1)):
            C_diff[f'{i}_{P}_{m}'] = cum_C[f'ssp534-sulfur_{i}_{P}_{m}'] - cum_C[f'ssp534-over_{i}_{P}_{m}']
            
            stat_list.append(round(C_diff[f'{i}_{P}_{m}'],1))
        
        mean_val = round(np.mean(stat_list), 1)
        main_list.append([P, i, round(C_diff[f'{i}_{P}_0'],1), 
                          round(C_diff[f'{i}_{P}_1'],1),
                          round(C_diff[f'{i}_{P}_{m}'],2),
                          round(np.mean(stat_list), 1)])
        
        print(mean_val)




land
[18.0, 12.4, 16.0]
15.5
ocean
[3.1, 2.2, 3.7]
3.0
total
[21.1, 14.6, 19.7]
18.5
land
[8.6, 6.2, 3.9]
6.2
ocean
[3.3, 0.8, 2.5]
2.2
total
[11.9, 7.1, 6.3]
8.4
land
[-13.6, -10.3, -11.8]
-11.9
ocean
[0.3, 2.8, 1.8]
1.6
total
[-13.3, -7.5, -10.0]
-10.3
land
[12.6, 7.6, 7.7]
9.3
ocean
[6.7, 5.8, 7.6]
6.7
total
[19.3, 13.4, 15.4]
16.0


In [23]:
dataf = pd.DataFrame(main_list, columns=['Phase', 'area', 'mem1', 'mem2', 'mem3', 'mean'])

In [24]:
dataf

,Phase,area,mem1,mem2,mem3,mean
0,I,land,18.0,12.4,16.0,15.5
1,I,ocean,3.1,2.2,3.7,3.0
2,I,total,21.1,14.6,19.7,18.5
3,II,land,8.6,6.2,3.9,6.2
4,II,ocean,3.3,0.8,2.5,2.2
5,II,total,11.9,7.1,6.3,8.4
6,III,land,-13.6,-10.3,-11.8,-11.9
7,III,ocean,0.3,2.8,1.8,1.6
8,III,total,-13.3,-7.5,-10.0,-10.3
9,all,land,12.6,7.6,7.7,9.3


In [25]:
dataf.to_csv('/data/scratch/globc/baur/NER_analysis/data/cumulative_C_diff.csv')